In [ ]:
import sys
username = 'admin' # 'admin' for BO1 lab
sys.path.append('c:/users/'+ username +'/miniconda3/lib/site-packages')
import qontrol
import os


# Setup Qontroller
serial_port_name = "COM3" #"/dev/tty.usbserial-FT06QAZ5" # name of the USB Serial Port
q = qontrol.QXOutput(serial_port_name = serial_port_name, response_timeout = 0.1)

print ("Qontroller '{:}' initialised with firmware {:} and {:} channels".format(q.device_id, q.firmware, q.n_chs) )

# Set voltage on each channel to its index in volts, read voltage, current

K = 0 # the channel connected to the device
V = 6.0 # the Voltage to be applied to channel K
I = 0
voltage_list = [] # list for subsequent voltage measurements (to be averaged)
current_list = [] # list for subsequent current measurements (to be averaged)
measured_voltage = 0
measured_current = 0
averaged_voltage = 0
averaged_current = 0
max_current = 50 # current compliance
max_voltage = 12 # voltage compliance
samples = 100 # number of measurements to be averaged



#Set current for channel K
#q.i[K] = I

# Set the compliances
for channel in range(q.n_chs):
    q.imax[channel] = max_current
    q.vmax[channel] = max_voltage
    
#Set voltage for channel K
q.v[K] = V



In [ ]:
for i in range(samples):
    
    measured_voltage = q.v[K]
    measured_current = q.i[K]
    voltage_list.append(measured_voltage)
    current_list.append(measured_current)
    averaged_voltage += measured_voltage
    averaged_current += measured_current

averaged_voltage /= samples
averaged_current /= samples
derived_resistance = (averaged_voltage/averaged_current)*1000

print("Channel {:} set to {:} V, measured {:} V and {:} mA\n".format(K, V, averaged_voltage, averaged_current))
print("The derived resistance is: {:} Ohm\n".format(derived_resistance))
print("Voltage list: \n", voltage_list)
print("Current list: \n", current_list)

# Open the log file
meas_file = open("Resistance_measurements_Q8b_L3_-5_-3_a_MZI.txt", "a")
if os.stat("Resistance_measurements_Q8b_L3_-5_-3_a_MZI.txt").st_size == 0:
    meas_file.write( "Channel,Voltage[V],Averaged_Voltage[V],Averaged_Current[mA],Derived_Resistance[Ohm]\n")

meas_file.write(str(K) + "," + str(V) + "," + str(averaged_voltage) + "," + str(averaged_current) + "," + str(derived_resistance)+ "\n")

# Close the log file
meas_file.close()

# Set all channels to zero
q.v[:] = 0
q.i[:] = 0

q.close()

#help(qontrol.QXOutput)